In [2]:
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma        ; kkma = Kkma()
from konlpy.tag import Hannanum    ; hannanum = Hannanum()
from konlpy.tag import Okt         ; t = Okt()     # 구 트위터
from konlpy.tag import *
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_excel('mergedtitle.xlsx')
df

,Unnamed: 0,Date,title,Price
0,0,20190101,Is it true that President Moon Jae-in's remark...,0
1,1,20190102,Samsung Display's investment expansion 'good n...,0
2,2,20190103,[Editorial] The KOSPI that collapsed in the 20...,1
3,3,20190104,"""Gangnam-do 2 billion won is too expensive"" Th...",1
4,4,20190105,"[Photo] BLACKPINK Jisoo, 'Chic Eyes' (Golden D...",0
...,...,...,...,...
1091,1091,20211227,"61 million won Bitcoin, 25 million won in the ...",1
1092,1092,20211228,"Ants sell Samsung Electronics 584 billion won,...",0
1093,1093,20211229,"""60% of serious accidents happen in the constr...",0
1094,1094,20211230,"Independent management of SK 'Intel NAND' ""We ...",0


In [ ]:
import re

okt = Okt()
n_ = []
title_rename = []
for i in range(len(df)):
  if(i % 10000 ==0):
    print(i,"단계 완료")
  title_rename.append(re.sub("[\(\[].*?[\)\]]", "",df.iloc[i]['title']))
  n_.append(' '.join(okt.nouns(df.iloc[i]['title'])))
df['nouns'] = n_
df['title']=title_rename
df = df[df['nouns']!='']

#2차 불용어 제거
df['title'] = df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['title'].replace('', np.nan, inplace=True)
df = df.dropna(how='any')

0 단계 완료


In [4]:
vocab = {}
cnt = 0
for i in df['nouns']:
    i = i.split(' ')
    for j in range(len(i)):
        if i[j] in vocab or len(i[j])<=1:
            cnt=cnt+1
            pass
        else:
            vocab[i[j]]=0
vocab
print(len(vocab), vocab)

21514 {'김현수': 0, '장관': 0, '탄소': 0, '중립': 0, '시대': 0, '농사': 0, '방식': 0, '세계': 0, '주방': 0, '밀폐': 0, '용기': 0, '대명사': 0, '이재': 0, '직접': 0, '일자리': 0, '채용': 0, '바이오': 0, '활황': 0, '모주': 0, '줄줄이': 0, '대기': 0, '유망': 0, '바이든': 0, '그린': 0, '친환경': 0, '신재생에너지': 0, '뉴욕증시': 0, '최고': 0, '마감': 0, '다우': 0, '천장': 0, '화이자': 0, '백신': 0, '긴급': 0, '사용': 0, '승인': 0, '한국': 0, '코로나': 0, '공급': 0, '개미': 0, '코스피': 0, '산타랠리': 0, '훌쩍': 0, '혼자': 0, '산다': 0, '소형': 0, '열풍': 0, '하남': 0, '시청역': 0, '해링턴': 0, '타워': 0, '센트럴': 0, '관심': 0, '모든': 0, '위기': 0, '기업': 0, '자영': 0, '업자': 0, '속보': 0, '연간': 0, '수출': 0, '지난해': 0, '선방': 0, '워낙': 0, '최악': 0, '전문직': 0, '주재': 0, '방문': 0, '교수': 0, '비자': 0, '개월': 0, '제한': 0, '단독': 0, '국민소득': 0, '백화점': 0, '직원': 0, '고객': 0, '화장실': 0, '일본차': 0, '점유': 0, '증발': 0, '닛산': 0, '인피니티': 0, '금융': 0, '감독': 0, '당국': 0, '수장': 0, '새해': 0, '화두': 0, '성수': 0, '정책금융': 0, '확대': 0, '액정': 0, '파손': 0, '최신': 0, '교체': 0, '고용노동부': 0, '테크': 0, '대리전': 0, '중국': 0, '전기차': 0, '스타트업': 0, '경쟁': 0, '재택근무': 0, '역설': 0, '작년': 0

In [5]:
df['주가변동'].value_counts()

1.0    32186
0.0    29724
Name: 주가변동, dtype: int64

In [6]:
up = 32186
down = 29724
up_ratio = up/(up+down)
down_ratio = down/(up+down)

import collections
for i,w in enumerate(df['nouns']):
    w = w.split(' ')
    if (df.iloc[i]['주가변동']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

In [7]:
sent_dictionary = vocab

In [31]:
len(sent_dictionary.keys())

21514

In [32]:
total = []
for i,w  in enumerate(df['nouns']):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in sent_dictionary):
          continue
        else:
          sent_score = sent_score + sent_dictionary[j]
    total.append(sent_score/len(w))
df['sent_score'] = total

In [33]:
df

,일자,제목,주가변동,nouns,sent_score
0,20210101,김현수 장관 탄소중립 시대 농사 방식 완전히 바꿔야,1.0,김현수 장관 탄소 중립 시대 농사 방식,3.169721
1,20210101,잠그고 또 잠근다 세계의 주방 꽉 잡은 밀폐용기 대명사,1.0,또 세계 주방 꽉 밀폐 용기 대명사,2.440617
2,20210101,이재갑 장관 직접일자리 만개 중 분기 내 채용,1.0,이재 갑 장관 직접 일자리 중 내 채용,2.607895
3,20210101,바이오팜 활황 맞은 공모주 대어급 줄줄이 대기,1.0,바이오 활황 모주 급 줄줄이 대기 유망 주,-7.588528
4,20210101,바이든 시대 그린가 뜬다 친환경 신재생에너지 담아라,1.0,바이든 시대 그린 친환경 신재생에너지 유망 주,1.940536
...,...,...,...,...,...
61906,20211231,허위 재무제표로 억 부당이익,0.0,허위 재무제표 부당 이익,-5.886044
61907,20211231,새만금 태양광 주민참여채권 억 발행 연 수익 설계,0.0,새만금 태양광 주민 참여 채권 발행 연 수익 설계,1.722368
61908,20211231,중앙디앤엠 자회사 에코엠넷 흡수합병 완료,0.0,중앙 디앤엠 자회사 에코 엠넷 흡수 합병 완료,0.215801
61909,20211231,현대오일뱅크 폐플라스틱 열분해유 넣어 친환경 납사 생산,0.0,뱅크 폐 플라스틱 열분해 유 친환경 납사 생산,1.949786


In [34]:
sum = 0
for i in range(len(vocab)):
  sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(df)):
  if(df.iloc[i]['sent_score']>sent_mean):
    a_.append(1)
  else:
    a_.append(0)
df['sent_label'] = a_

In [63]:
df['일자'].value_counts()

20210908    411
20210128    328
20210428    325
20210420    293
20210427    293
           ... 
20211016     14
20210102     14
20211030     13
20210320     11
20210123      8
Name: 일자, Length: 365, dtype: int64

In [64]:
df1 = df['일자'].value_counts()

In [65]:
df1 = df1.sort_index()
df1

20210101     75
20210102     14
20210103     93
20210104    270
20210105    261
           ... 
20211227    193
20211228    190
20211229    181
20211230    191
20211231    110
Name: 일자, Length: 365, dtype: int64

In [66]:
df1 = pd.DataFrame(df1)

# df2 = df1.rename(columns={'일자': '횟수'})
df1.reset_index(inplace=True)
# df2.rename(index='일자')
# # df2 = df2.rename(columns={'index': '일자'})

In [67]:
df1

,index,일자
0,20210101,75
1,20210102,14
2,20210103,93
3,20210104,270
4,20210105,261
...,...,...
360,20211227,193
361,20211228,190
362,20211229,181
363,20211230,191


In [61]:
df1.columns = ['일자', '횟수']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements

In [60]:
df1.set_index('일자')
df1

KeyError: "None of ['일자'] are in the columns"

In [41]:
df.to_pickle('sent_data.pkl')

In [42]:
df.to_csv('sent_score_whole.csv', index=True, encoding='utf-8')
df

,일자,제목,주가변동,nouns,sent_score,sent_label
0,20210101,김현수 장관 탄소중립 시대 농사 방식 완전히 바꿔야,1.0,김현수 장관 탄소 중립 시대 농사 방식,3.169721,1
1,20210101,잠그고 또 잠근다 세계의 주방 꽉 잡은 밀폐용기 대명사,1.0,또 세계 주방 꽉 밀폐 용기 대명사,2.440617,1
2,20210101,이재갑 장관 직접일자리 만개 중 분기 내 채용,1.0,이재 갑 장관 직접 일자리 중 내 채용,2.607895,1
3,20210101,바이오팜 활황 맞은 공모주 대어급 줄줄이 대기,1.0,바이오 활황 모주 급 줄줄이 대기 유망 주,-7.588528,0
4,20210101,바이든 시대 그린가 뜬다 친환경 신재생에너지 담아라,1.0,바이든 시대 그린 친환경 신재생에너지 유망 주,1.940536,1
...,...,...,...,...,...,...
61906,20211231,허위 재무제표로 억 부당이익,0.0,허위 재무제표 부당 이익,-5.886044,0
61907,20211231,새만금 태양광 주민참여채권 억 발행 연 수익 설계,0.0,새만금 태양광 주민 참여 채권 발행 연 수익 설계,1.722368,1
61908,20211231,중앙디앤엠 자회사 에코엠넷 흡수합병 완료,0.0,중앙 디앤엠 자회사 에코 엠넷 흡수 합병 완료,0.215801,1
61909,20211231,현대오일뱅크 폐플라스틱 열분해유 넣어 친환경 납사 생산,0.0,뱅크 폐 플라스틱 열분해 유 친환경 납사 생산,1.949786,1


In [54]:
df.to_excel('sent_score_whole.xlsx', index=True, encoding='utf-8')
df

,일자,제목,주가변동,nouns,sent_score,sent_label
0,20210101,김현수 장관 탄소중립 시대 농사 방식 완전히 바꿔야,1.0,김현수 장관 탄소 중립 시대 농사 방식,3.169721,1
1,20210101,잠그고 또 잠근다 세계의 주방 꽉 잡은 밀폐용기 대명사,1.0,또 세계 주방 꽉 밀폐 용기 대명사,2.440617,1
2,20210101,이재갑 장관 직접일자리 만개 중 분기 내 채용,1.0,이재 갑 장관 직접 일자리 중 내 채용,2.607895,1
3,20210101,바이오팜 활황 맞은 공모주 대어급 줄줄이 대기,1.0,바이오 활황 모주 급 줄줄이 대기 유망 주,-7.588528,0
4,20210101,바이든 시대 그린가 뜬다 친환경 신재생에너지 담아라,1.0,바이든 시대 그린 친환경 신재생에너지 유망 주,1.940536,1
...,...,...,...,...,...,...
61906,20211231,허위 재무제표로 억 부당이익,0.0,허위 재무제표 부당 이익,-5.886044,0
61907,20211231,새만금 태양광 주민참여채권 억 발행 연 수익 설계,0.0,새만금 태양광 주민 참여 채권 발행 연 수익 설계,1.722368,1
61908,20211231,중앙디앤엠 자회사 에코엠넷 흡수합병 완료,0.0,중앙 디앤엠 자회사 에코 엠넷 흡수 합병 완료,0.215801,1
61909,20211231,현대오일뱅크 폐플라스틱 열분해유 넣어 친환경 납사 생산,0.0,뱅크 폐 플라스틱 열분해 유 친환경 납사 생산,1.949786,1


In [55]:
df1 = df.groupby(['일자']).sum()

In [58]:
df1

,주가변동,sent_score,sent_label
일자,,,
20210101,75.0,230.606418,57
20210102,14.0,27.511624,11
20210103,93.0,243.469208,71
20210104,270.0,125.028069,175
20210105,0.0,-327.444465,124
...,...,...,...
20211227,193.0,-33.983774,113
20211228,0.0,-182.494010,96
20211229,0.0,-268.612711,77


In [47]:
df_new

NameError: name 'df_new' is not defined

In [57]:
# df2.set_index('일자')

df1['mean_sent_score']= df1['sent_score'] / df1['횟수']
df1

KeyError: '횟수'

In [ ]:
label_mean = df_new['mean_sent_score'].mean()

In [ ]:
a_ = []
for i in range(365):
  if(df_new.iloc[i]['mean_sent_score']>label_mean):
    a_.append(1)
  else:
    a_.append(0)
df_new['sent_label_whole'] = a_

In [52]:
df_new

,일자,제목,주가변동_x,nouns,sent_score_x,sent_label_x,주가변동_y,sent_score_y,sent_label_y
0,20210101,김현수 장관 탄소중립 시대 농사 방식 완전히 바꿔야,1.0,김현수 장관 탄소 중립 시대 농사 방식,3.169721,1,75.0,230.606418,57
1,20210101,잠그고 또 잠근다 세계의 주방 꽉 잡은 밀폐용기 대명사,1.0,또 세계 주방 꽉 밀폐 용기 대명사,2.440617,1,75.0,230.606418,57
2,20210101,이재갑 장관 직접일자리 만개 중 분기 내 채용,1.0,이재 갑 장관 직접 일자리 중 내 채용,2.607895,1,75.0,230.606418,57
3,20210101,바이오팜 활황 맞은 공모주 대어급 줄줄이 대기,1.0,바이오 활황 모주 급 줄줄이 대기 유망 주,-7.588528,0,75.0,230.606418,57
4,20210101,바이든 시대 그린가 뜬다 친환경 신재생에너지 담아라,1.0,바이든 시대 그린 친환경 신재생에너지 유망 주,1.940536,1,75.0,230.606418,57
...,...,...,...,...,...,...,...,...,...
61905,20211231,허위 재무제표로 억 부당이익,0.0,허위 재무제표 부당 이익,-5.886044,0,0.0,-4.206881,64
61906,20211231,새만금 태양광 주민참여채권 억 발행 연 수익 설계,0.0,새만금 태양광 주민 참여 채권 발행 연 수익 설계,1.722368,1,0.0,-4.206881,64
61907,20211231,중앙디앤엠 자회사 에코엠넷 흡수합병 완료,0.0,중앙 디앤엠 자회사 에코 엠넷 흡수 합병 완료,0.215801,1,0.0,-4.206881,64
61908,20211231,현대오일뱅크 폐플라스틱 열분해유 넣어 친환경 납사 생산,0.0,뱅크 폐 플라스틱 열분해 유 친환경 납사 생산,1.949786,1,0.0,-4.206881,64


In [ ]:
df1

In [51]:
df_new = pd.merge(df,df1,how='outer', on='일자')

In [ ]:
df

In [ ]:
df_new.to_csv('sent_score_new.csv', index=True, encoding='euc-kr')

In [ ]:
df['sent_label_whole'].value_counts()

In [3]:
df = pd.read_csv('KOSPInews-en.csv')
df

,Date,title,stock price fluctuations
0,20210101,"[Book Village] Corporate personal ""Let's prepa...",1
1,20210102,"Suspension of Harimgak, a famous Chinese resta...",1
2,20210103,Only 9 out of 20 companies that have reduced g...,1
3,20210104,"Bernanke, ""The US economy recovers faster than...",1
4,20210105,"The government releases 180,000 tons of stockp...",0
...,...,...,...
360,20211227,"61 million won Bitcoin, 25 million won in the ...",1
361,20211228,"Ants sell Samsung Electronics 584 billion won,...",0
362,20211229,"""60% of serious accidents happen in the constr...",0
363,20211230,"Independent management of SK 'Intel NAND' ""We ...",0


In [5]:
df1 = pd.read_csv('sent_score_new.csv', encoding="euc-kr")
df1

,Unnamed: 0,일자,KOSPI,sent_score,sent_label,뉴스개수,mean_sent_score,sent_label_whole
0,0,20210101,75,230.606418,57,75,3.074752,1
1,1,20210102,14,27.511624,11,14,1.965116,1
2,2,20210103,93,243.469208,71,93,2.617948,1
3,3,20210104,270,125.028069,175,270,0.463067,1
4,4,20210105,0,-327.444465,124,261,-1.254576,0
...,...,...,...,...,...,...,...,...
360,360,20211227,193,-33.983774,113,193,-0.176082,1
361,361,20211228,0,-182.494010,96,190,-0.960495,0
362,362,20211229,0,-268.612711,77,181,-1.484048,0
363,363,20211230,0,-235.003238,92,191,-1.230383,0


In [9]:
df

,Date,title,stock price fluctuations
0,20210101,"[Book Village] Corporate personal ""Let's prepa...",1
1,20210102,"Suspension of Harimgak, a famous Chinese resta...",1
2,20210103,Only 9 out of 20 companies that have reduced g...,1
3,20210104,"Bernanke, ""The US economy recovers faster than...",1
4,20210105,"The government releases 180,000 tons of stockp...",0
...,...,...,...
360,20211227,"61 million won Bitcoin, 25 million won in the ...",1
361,20211228,"Ants sell Samsung Electronics 584 billion won,...",0
362,20211229,"""60% of serious accidents happen in the constr...",0
363,20211230,"Independent management of SK 'Intel NAND' ""We ...",0


In [12]:
df['sent_score']=df1['sent_label_whole']

In [13]:
df

,Date,title,stock price fluctuations,sent_score
0,20210101,"[Book Village] Corporate personal ""Let's prepa...",1,1
1,20210102,"Suspension of Harimgak, a famous Chinese resta...",1,1
2,20210103,Only 9 out of 20 companies that have reduced g...,1,1
3,20210104,"Bernanke, ""The US economy recovers faster than...",1,1
4,20210105,"The government releases 180,000 tons of stockp...",0,0
...,...,...,...,...
360,20211227,"61 million won Bitcoin, 25 million won in the ...",1,1
361,20211228,"Ants sell Samsung Electronics 584 billion won,...",0,0
362,20211229,"""60% of serious accidents happen in the constr...",0,0
363,20211230,"Independent management of SK 'Intel NAND' ""We ...",0,0


In [15]:
df.to_csv('sent_score_new_en.csv', index=True, encoding='utf-8')